# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (можно брать только часть текста, если считается слишком долго). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг \<start>  
    - можете использовать тот же подход с матрицей вероятностей, но по строкам хронить биграмы, а по колонкам униграммы 
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)
    - у вас будут словари с индексами биграммов и униграммов, не перепутайте их при переводе индекса в слово - словарь биграммов будет больше словаря униграммов и все индексы из униграммного словаря будут формально подходить для словаря биграммов (не будет ошибки при id2bigram[unigram_id]), но маппинг при этом будет совершенно неправильным 

In [1]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter
from nltk.tokenize import sent_tokenize



In [2]:
with open("2ch_corpus.txt", "r") as f:
    dvach = f.read()

In [3]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

Длина корпуса в предложениях

In [4]:
len(sent_tokenize(dvach))

170565

In [5]:
train, test = sent_tokenize(dvach)[:170500], sent_tokenize(dvach)[170500:]

In [6]:
len(test)

65

In [8]:
def ngrammer(tokens, n=3):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams


def n_markov_assumption_model(text, ngramms = 3):

    text = [['<start>']*(ngramms-1) + normalize(sent) + ['<end>'] for sent in text]

    unigrams = Counter()
    bigrams = Counter()
    trigrams = Counter()

    for sent in text:
        unigrams.update(sent)
        bigrams.update(ngrammer(sent, ngramms-1))
        trigrams.update(ngrammer(sent, ngramms))

    return unigrams, bigrams, trigrams


def compute_joint_proba_markov_assumption(text, unigram_counts, bigram_counts, trigram_counts, ngramms = 3):
    prob = 0
    tokens = normalize(text)
    for ngram in ngrammer(['<start>']*(ngramms-1) + tokens + ['<end>'], ngramms):
        word1, word2, word3 = ngram.split()
        bigramm = " ".join([word1, word2])
        if bigramm in bigram_counts and ngram in trigram_counts:
            prob += np.log(trigram_counts[ngram]/bigram_counts[bigramm])
        else:
            prob += np.log(2e-5)
    
    return prob, len(tokens)

In [9]:
unigrams, bigrams, trigrams = n_markov_assumption_model(train)

In [10]:
phrase = 'Безграмотное быдло с дубляжом, войсовером, порнографией и котиками '

compute_joint_proba_markov_assumption(phrase, unigrams, bigrams, trigrams)

(-22.170201969452712, 8)

In [11]:
def perplexity(logp, N):
    return np.exp((-1/N) * logp)


ppl = []

for text in test:

    ppl.append(perplexity(*compute_joint_proba_markov_assumption(text, unigrams, bigrams, trigrams)))

print(f"Perplexity of the model is {np.array(ppl).mean()}")

Perplexity of the model is 38756182.74456518


In [12]:
from scipy.sparse import dok_matrix, lil_matrix

In [14]:
matrix_dvach = lil_matrix((len(bigrams), 
                         len(unigrams)))

id2word_unigrams = list(unigrams)
id2word_bigrams = list(bigrams)
word2id_unigrams = {word:i for i, word in enumerate(id2word_unigrams)}
word2id_bigrams = {word:i for i, word in enumerate(id2word_bigrams)}

for ngram in trigrams:
    word1, word2, word3 = ngram.split()
    bigramm = " ".join([word1, word2])
    
    matrix_dvach[word2id_bigrams[bigramm], word2id_unigrams[word3]] =  (trigrams[ngram]/
                                                                     bigrams[bigramm])


In [32]:
def generate(matrix, id2word_bigrams, word2id_bigrams, id2word_unigrams, word2id_unigrams, n=100, start='<start> <start>'):
    text = []
    current_idx = word2id_bigrams[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx].toarray()[0])

        chosen_word = id2word_unigrams[chosen]
        
        text.append(chosen_word)

        last_bigramm = id2word_bigrams[current_idx]

        word1, word2 = last_bigramm.split()

        chosen = word2id_bigrams[" ".join([word2, chosen_word])]
        
        if chosen_word == '<end>':
            chosen = word2id_bigrams['<start> <start>']
        
        current_idx = chosen
    
    return ' '.join(text)

In [36]:
print(generate(matrix_dvach, id2word_bigrams, word2id_bigrams, id2word_unigrams, word2id_unigrams).replace("<end>", "\n"))

дабы сохранить все что ты горишь дома у каждого второго вижу моллеманькой либо миллисентом ебучим 
 бюджет опять таки за счет того что синдзи превратил весь мир 
 садитесь в пативен картина называется максим тесак 
 на разные фракции 
 спасибо бро поддержал 
 podrasti suka bezmozglaya 
 пояснение да есть такой реальный персонаж кто здесь недавно примерно так же постройки и арефакты фактически такие же отшибленные и будут выебываться как утерли нос американцам сукаааааа 
 воспитала мать и дочь 
 популярные сюжеты в кино-книгах одни и никто ничего не растет или растет медленней ввп то их поведение никак не


## Задание № 2* (2 балла). 

Измените функцию generate_with_beam_search так, чтобы она работала с моделью, которая учитывает два предыдущих слова. 
Сравните получаемый результат с первым заданием. 
Также попробуйте начинать генерацию не с нуля (подавая \<start> \<start>), а с какого-то промпта. Но помните, что учитываться будут только два последних слова, так что не делайте длинные промпты.

In [43]:
# сделаем класс чтобы хранить каждый из лучей
class Beam:
    def __init__(self, sequence: list, score: float):
        self.sequence: list = sequence
        self.score: float = score 

##### Здесь, правда, теперь наоборот получается уж слишком детерминированное решение в любом случае

In [127]:

def generate_with_beam_search(matrix, id2word_bigramm, word2id_bigramm, id2word_unigramm, word2id_unigramm, n=50, max_beams=7, start='<start> <start>'):
    # изначально у нас один луч с заданным началом (start по дефолту)
    initial_node = Beam(sequence=start.split(), score=np.log1p(0))
    beams = [initial_node]
    
    for i in range(n):
        # делаем n шагов генерации
        new_beams = []
        # на каждом шаге продолжаем каждый из имеющихся лучей
        for beam in beams:
            # лучи которые уже закончены не продолжаем (но и не удаляем)
            if beam.sequence[-1] == '<end>':
                new_beams.append(beam)
                continue
            
            # наша языковая модель предсказывает на основе предыдущего слова
            # достанем его из beam.sequence
            
            last_id = word2id_bigramm[" ".join(beam.sequence[-2:])]
            
            # посмотрим вероятности продолжений для предыдущего слова
            probas = matrix[last_id].toarray()[0]
            
            # возьмем топ самых вероятных продолжений
            top_idxs = probas.argsort()[:-(max_beams+1):-1]
            for top_id in top_idxs:
                # иногда вероятности будут нулевые, такое не добавляем
                if not probas[top_id]:
                    break
                
                # создадим новый луч на основе текущего и варианта продолжения
                new_sequence = beam.sequence + [id2word_unigramm[top_id]]
                # скор каждого луча это произведение вероятностей (или сумма логарифмов)
                new_score = beam.score + np.log1p(probas[top_id])
                new_beam = Beam(sequence=new_sequence, score=new_score)
                new_beams.append(new_beam)
        # отсортируем лучи по скору и возьмем только топ max_beams
        beams = sorted(new_beams, key=lambda x: x.score, reverse=True)[:max_beams]

    
    # в конце возвращаем случайный луч, чтобы алгоритм хотя бы на кототких последовательностях был чуть менее детерминированным
    best_sequence = np.random.choice(beams).sequence

    
    return ' '.join(best_sequence)

In [111]:
print(generate_with_beam_search(matrix_dvach, id2word_bigrams, word2id_bigrams, id2word_unigrams, word2id_unigrams).replace("<start> <start>", ""))

 ну и конечно же там друг возле друга близков случае чего кричат дурным голосом в рацию о подмоге чтобы прибежал мужик и по сей день считаю кровавые деньги лучшей частью а сан хорошую пушку со жмура теперь и кинамана побуждают к сотрудничеству первыми открывать огонь после того как я понял


In [135]:
print(generate_with_beam_search(matrix_dvach, id2word_bigrams, word2id_bigrams, id2word_unigrams, word2id_unigrams, start = "чего только", n=5).replace("<start> <start>", ""))

чего только стоит слияние руби и джаву


In [137]:
print(generate_with_beam_search(matrix_dvach, id2word_bigrams, word2id_bigrams, id2word_unigrams, word2id_unigrams, start = "чего только", n=5).replace("<start> <start>", ""))

чего только стоит js 100 часов лол


In [150]:
print(generate_with_beam_search(matrix_dvach, id2word_bigrams, word2id_bigrams, id2word_unigrams, word2id_unigrams, start = "когда он пришел", n=10).replace("<start> <start>", ""))

когда он пришел в приемную партии « единая россия партия дела <end>
